In [1]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     |████████████████████████████████| 9.1 MB 5.3 MB/s 
     |████████████████████████████████| 4.3 MB 37.1 MB/s 
     |████████████████████████████████| 180 kB 49.7 MB/s 
     |████████████████████████████████| 111 kB 50.4 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 178 kB 49.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 125 kB 48.6 MB/s 
     |████████████████████████████████| 790 kB 49.2 MB/s 
     |████████████████████████████████| 374 kB 52.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.22 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.5.0 which is incompatible.
google-colab 1.0.0 requires ipytho

In [22]:
%%writefile predict.py
import streamlit as st
import keras
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

st.title("BitGo - Cryptocurency trend Prediction")

ADAmodel = keras.models.load_model("ADAmodel.hdf5")
BNBmodel = keras.models.load_model("BNBmodel.hdf5")
BTCmodel = keras.models.load_model("BTCmodel.hdf5")
ETHmodel = keras.models.load_model("ETHmodel.hdf5")


endpointADA = 'https://min-api.cryptocompare.com/data/histoday'
resADA = requests.get(endpointADA + '?fsym=ADA&tsym=USD&limit=700')
dfADA = pd.DataFrame(json.loads(resADA.content)['Data'])
dfADA = dfADA.set_index('time')
dfADA.index = pd.to_datetime(dfADA.index, unit='s')
target_col_ADA='close'

endpointBNB = 'https://min-api.cryptocompare.com/data/histoday'
resBNB = requests.get(endpointBNB + '?fsym=BNB&tsym=USD&limit=500')
dfBNB = pd.DataFrame(json.loads(resBNB.content)['Data'])
dfBNB = dfBNB.set_index('time')
dfBNB.index = pd.to_datetime(dfBNB.index, unit='s')
target_col_BNB='close'

endpointBTC = 'https://min-api.cryptocompare.com/data/histoday'
resBTC = requests.get(endpointBTC + '?fsym=BTC&tsym=USD&limit=2000')
dfBTC = pd.DataFrame(json.loads(resBTC.content)['Data'])
dfBTC = dfBTC.set_index('time')
dfBTC.index = pd.to_datetime(dfBTC.index, unit='s')
target_col_BTC='close'

endpointETH = 'https://min-api.cryptocompare.com/data/histoday'
resETH = requests.get(endpointETH + '?fsym=ETH&tsym=USD&limit=2000')
dfETH = pd.DataFrame(json.loads(resETH.content)['Data'])
dfETH = dfETH.set_index('time')
dfETH.index = pd.to_datetime(dfETH.index, unit='s')
target_col_ETH='close'




dfADA = dfADA.drop(['conversionType','conversionSymbol'], axis=1)

dfBNB = dfBNB.drop(['conversionType','conversionSymbol'], axis=1)

dfBTC = dfBTC.drop(['conversionType','conversionSymbol'], axis=1)

dfETH = dfETH.drop(['conversionType','conversionSymbol'], axis=1)






def train_test_split(df, test_size=0.4):
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data

def normalise_zero_base(df):
    return df / df.iloc[0] - 1

def normalise_min_max(df):
    return (df - df.min()) / (data.max() - df.min())

def extract_window_data(df, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

def prepare_data(df, target_col, window_len=5, zero_base=True, test_size=0.4):
    train_data, test_data = train_test_split(df, test_size=test_size)
    X_train = extract_window_data(train_data, window_len, zero_base)
    X_test = extract_window_data(test_data, window_len, zero_base)
    y_train = train_data[target_col][window_len:].values
    y_test = test_data[target_col][window_len:].values
    if zero_base:
        y_train = y_train / train_data[target_col][:-window_len].values - 1
        y_test = y_test / test_data[target_col][:-window_len].values - 1

    return train_data, test_data, X_train, X_test, y_train, y_test



np.random.seed(42)
window_len = 5
test_size = 0.4
zero_base = True

trainADA, testADA, X_trainADA, X_testADA, y_trainADA, y_testADA = prepare_data(dfADA, target_col_ADA, window_len=window_len, zero_base=zero_base, test_size=test_size)
trainETH, testETH, X_trainETH, X_testETH, y_trainETH, y_testETH = prepare_data(dfETH, target_col_ETH, window_len=window_len, zero_base=zero_base, test_size=test_size)
trainbtc, testbtc, X_trainbtc, X_testbtc, y_trainbtc, y_testbtc = prepare_data(dfBTC, target_col_BTC, window_len=window_len, zero_base=zero_base, test_size=test_size)
trainbnb, testbnb, X_trainbnb, X_testbnb, y_trainbnb, y_testbnb = prepare_data(dfBNB, target_col_BNB, window_len=window_len, zero_base=zero_base, test_size=test_size)


targetsADA = testADA[target_col_ADA][window_len:]
predADA = ADAmodel.predict(X_testADA).squeeze()
predADA = testADA[target_col_ADA].values[:-window_len] * (predADA + 1)
predADA = pd.Series(index=targetsADA.index, data=predADA)

targetsETH = testETH[target_col_ETH][window_len:]
predETH = ETHmodel.predict(X_testETH).squeeze()
predETH = testETH[target_col_ETH].values[:-window_len] * (predETH + 1)
predETH = pd.Series(index=targetsETH.index, data=predETH)

targetsbtc = testbtc[target_col_BTC][window_len:]
predbtc = BTCmodel.predict(X_testbtc).squeeze()
predbtc = testbtc[target_col_BTC].values[:-window_len] * (predbtc + 1)
predbtc = pd.Series(index=targetsbtc.index, data=predbtc)

targetsbnb = testbnb[target_col_BNB][window_len:]
predbnb = BNBmodel.predict(X_testbnb).squeeze()
predbnb = testbnb[target_col_BNB].values[:-window_len] * (predbnb + 1)
predbnb = pd.Series(index=targetsbnb.index, data=predbnb)

crypto = ('BTC','BNB','ETH','ADA')
user_input = st.selectbox("Select Crypto to predict",crypto)

if user_input == 'BTC':
    st.subheader('Full data from last few days')
    st.write(dfBTC.tail())
    st.subheader('Prediction')
    fig = plt.figure(figsize=(12,6))
    plt.plot(predbtc, label='Closing Price', linewidth=2)
    plt.xlabel('Time(Months)', fontsize=14)
    plt.ylabel('price [CAD]', fontsize=14)
    plt.legend(loc='best', fontsize=16)
    st.pyplot(fig)
   
    st.subheader("Graph of Volume Traded VS Closing Price (y axis values are e+10)")
    close = list(dfBTC.close)
    date = list(dfBTC.volumeto)
    fig2 = plt.figure(figsize = (10, 5))
    # creating the bar plot
    plt.bar(close, date, linewidth=3, color='red')
    plt.xlabel("closing price(in $)")
    plt.ylabel("closing volume")
    st.pyplot(fig2)

if user_input == 'BNB':
    st.subheader('Full data from last few days')
    st.write(dfBNB.tail())
    st.subheader('Prediction')
    fig = plt.figure(figsize=(12,6))
    plt.plot(predbnb, label='Closing Price', linewidth=2)
    plt.xlabel('Time(Months)', fontsize=14)
    plt.ylabel('price [CAD]', fontsize=14)
    plt.legend(loc='best', fontsize=16)
    st.pyplot(fig)

    st.subheader("Graph of Volume Traded VS Closing Price (y axis values are e+9)")
    close = list(dfBNB.close)
    date = list(dfBNB.volumeto)
    fig2 = plt.figure(figsize = (10, 5))
    # creating the bar plot
    plt.bar(close, date, linewidth=3, color='red')
    plt.xlabel("closing price(in $)")
    plt.ylabel("closing volume")
    st.pyplot(fig2)

  
if user_input == 'ETH':
    st.subheader('Full data from last few days')
    st.write(dfETH.tail())
    st.subheader('Prediction')
    fig = plt.figure(figsize=(12,6))
    plt.plot(predETH, label='Closing Price', linewidth=2)
    plt.xlabel('Time(Months)', fontsize=14)
    plt.ylabel('price [CAD]', fontsize=14)
    plt.legend(loc='best', fontsize=16)
    st.pyplot(fig)

    st.subheader("Graph of Volume Traded VS Closing Price (y axis values are e+10)")
    close = list(dfETH.close)
    date = list(dfETH.volumeto)
    fig2 = plt.figure(figsize = (10, 5))
    # creating the bar plot
    plt.bar(close, date, linewidth=3, color='red')
    plt.xlabel("closing price(in $)")
    plt.ylabel("closing volume")
    st.pyplot(fig2)


  

if user_input == 'ADA':
    st.subheader('Full data from last few days')
    st.write(dfADA.tail())
    st.subheader('Prediction')
    fig = plt.figure(figsize=(12,6))
    plt.plot(predADA, label='Closing Price', linewidth=2)
    plt.xlabel('Time(Months)', fontsize=14)
    plt.ylabel('price [CAD]', fontsize=14)
    plt.legend(loc='best', fontsize=16)
    st.pyplot(fig)

    st.subheader("Graph of Volume Traded VS Closing Price (y axis values are e+9)")
    close = list(dfADA.close)
    date = list(dfADA.volumeto)
    fig2 = plt.figure(figsize = (10, 5))
    # creating the bar plot
    plt.bar(close, date, linewidth=3, color='red')
    plt.xlabel("closing price(in $)")
    plt.ylabel("closing volume")
    st.pyplot(fig2)
    







Overwriting predict.py


In [3]:
!nohup streamlit run predict.py &
url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://adc4-34-72-156-224.ngrok.io'